In [1]:
# from https://jupyter-notebook.readthedocs.io/en/stable/examples/Notebook/Importing%20Notebooks.html

import io, os, sys, types

from IPython import get_ipython
from nbformat import read
from IPython.core.interactiveshell import InteractiveShell

def find_notebook(fullname, path=None):
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path_pure = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path_pure):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path_pure.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path
        nb_path = nb_path_pure.replace("_", "-")
        if os.path.isfile(nb_path):
            return nb_path

class NotebookLoader(object):
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod
    
class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]
sys.meta_path.append(NotebookFinder())

In [2]:
import numpy as np
import random
import torch
seed = 2023
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

In [3]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '4'

In [4]:
from pegon_utils import CTCTrainer
from models import *
model = 'ctc-crnn-no-stretch-v2/2023-05-24 18:24:32.874318.annot.ft.25ep.pt'
model_path = os.path.join('/workspace/ctc-crnn/', model)
model_type = os.path.split(os.path.split(model_path)[0])[-1]
model = torch.load(model_path)

In [5]:
from pegon_utils import FilenameOCRDataset
from torch.utils.data import DataLoader, Dataset
from torchvision.transforms import transforms
from torch.utils.data import random_split, ConcatDataset

dataset_transforms = {'ctc-crnn':
                      transforms.Compose([
                          transforms.Grayscale(num_output_channels=1),
                          transforms.Resize((model.image_height, model.image_width)),
                          transforms.RandomHorizontalFlip(p=1),
                          transforms.ToTensor()]),
                      'ctc-ftex-rnn':
                      transforms.Compose([
                          transforms.Resize((model.image_height, model.image_width)),
                          transforms.RandomHorizontalFlip(p=1),
                          transforms.ToTensor()]),
                      'ctc-ftex-no-rnn':
                      transforms.Compose([
                          transforms.Resize((model.image_height, model.image_width)),
                          transforms.RandomHorizontalFlip(p=1),
                          transforms.ToTensor()]),
                      'ctc-crnn-no-stretch':
                      transforms.Compose([
                          transforms.Grayscale(num_output_channels=1),
                          ResizeAndPadHorizontal(target_h=model.image_height, target_w=model.image_width),
                          transforms.RandomHorizontalFlip(p=1),
                          transforms.ToTensor(),]),
                      'ctc-crnn-no-stretch-v2':
                      transforms.Compose([
                          transforms.Grayscale(num_output_channels=1),
                          ResizeAndPadHorizontal(target_h=model.image_height, target_w=model.image_width),
                          transforms.RandomHorizontalFlip(p=1),
                          transforms.ToTensor(),
                      ])}[model_type]

annotated_dataset = FilenameOCRDataset('/workspace/Dataset/line_images_yolo',
                                       image_transform=dataset_transforms)

In [6]:
from jiwer import wer, cer
from itertools import starmap
from tqdm.notebook import tqdm

def evaluate(decoder, dataloader):
    char_map = decoder.alphabet
    
    def decode(arr):
        try:
            return ''.join(list(map(lambda x:char_map[x], arr)))
        except IndexError as e:
            e.args += (arr, f"Char map length: {len(char_map)}")
            raise
    res = []
    for image, filename in tqdm(dataloader):
        res.append((decoder.infer(image.to('cuda')), filename))
    return res

In [7]:
from pegon_utils import CTCDecoder, BestPathDecoder
from pegon_utils import PEGON_CHARS, CHAR_MAP
from pegon_utils import ctc_collate_fn

dataloader = DataLoader(annotated_dataset)
decoder = BestPathDecoder.from_path(model_path, CHAR_MAP, blank_char=PEGON_CHARS[0])

res = [(predictions[0], filenames[0]) for predictions, filenames in evaluate(decoder, dataloader)]

with open('res.csv', 'w') as f:
    for pred, filename in res:
        print(f'{filename};{pred}', file=f)

  0%|          | 0/44 [00:00<?, ?it/s]